<a href="https://colab.research.google.com/github/TheSoloLeveling/HuggingFace-BERT-Only/blob/main/HuggingFace_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


In [ ]:
from torch import nn, Tensor
import torch
from typing import List, Optional, Tuple, Union
import math
from collections import OrderedDict
import inspect
from typing import Callable, List, Optional, Set, Tuple, Union
from dataclasses import dataclass

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive')

In [ ]:
from generic import ModelOutput

In [ ]:
from transformers import BertConfig, BertTokenizer
from torch.utils.data import DataLoader
from datasets import load_dataset

# Load a dataset
dataset = load_dataset('imdb', split='train')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Load a pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

sliced_dataset = dataset.select(range(100))
tokenized_dataset = sliced_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [ ]:
tokenized_dataset[0]["text"]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
class BertEmbeddings(nn.Module):

  def __init__(self, config):

    super().__init__()
    self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)                 # hidden_size : dimension of encoder layer input
    self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)                                # max_position_embeddings = maximum sequennce length
    self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)                                        # token_type_embeddings : for having multiple sequences input for a task
    self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
    self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)), persistent=False)         # not updated during training
    self.register_buffer("token_type_ids", torch.zeros(self.position_ids.size(), dtype=torch.long), persistent=False)            # not updated during training

  def forward(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    token_type_ids: Optional[torch.LongTensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    past_key_values_length: int = 0,
  ) -> torch.Tensor:

    if input_ids is not None:
        input_shape = input_ids.size()
    else:
        input_shape = inputs_embeds.size()[:-1]

    seq_length = input_shape[1]      # input shape(1, 512)
    #print(input_shape)

    if position_ids is None:
        position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]          # position_ids[:, 0 : 512 + 0]

    #print("position_ids : position_ids[:, "+str(past_key_values_length)+" : "+str(seq_length) + " + "+ str(past_key_values_length) +"]")
    #print(position_ids)
    # Setting the token_type_ids to the registered buffer in constructor where it is all zeros, which usually occurs
    # when its auto-generated, registered buffer helps users when tracing the model without passing token_type_ids, solves
    # issue #5664
    if token_type_ids is None:
        if hasattr(self, "token_type_ids"):
            buffered_token_type_ids = self.token_type_ids[:, :seq_length]
            #print(buffered_token_type_ids.shape)
            buffered_token_type_ids_expanded = buffered_token_type_ids.expand(input_shape[0], seq_length)
            token_type_ids = buffered_token_type_ids_expanded
            #print(token_type_ids.shape)
        else:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

    if inputs_embeds is None:
        inputs_embeds = self.word_embeddings(input_ids)
        print("inputs_embeds : " +  str(inputs_embeds) + ",  shape : " + str(inputs_embeds.shape))

    token_type_embeddings = self.token_type_embeddings(token_type_ids)

    print("token_type_embeddings : " +  str(token_type_embeddings) + ",  shape : " + str( token_type_embeddings.shape))
    embeddings = inputs_embeds + token_type_embeddings
    if self.position_embedding_type == "absolute":
        position_embeddings = self.position_embeddings(position_ids)
        embeddings += position_embeddings
    embeddings = self.LayerNorm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings


In [ ]:
config = BertConfig()
myEmbeddings = BertEmbeddings(config)

In [ ]:
t = [101,
 1045,
 12524,
 1045,
 2572,
 8025,
 1011,
 3756,
 2013,
 2026,
 2678,
 3573,
 2138,]

In [ ]:
ids = torch.tensor(t)
ids = ids.unsqueeze(0)
print(ids.shape)
em = myEmbeddings(ids)

torch.Size([1, 13])
inputs_embeds : tensor([[[-0.6648,  0.4162,  0.5075,  ...,  0.0945,  1.5146,  0.3791],
         [-0.6925, -0.9747,  0.6626,  ...,  1.2095,  1.4783,  0.7562],
         [ 1.4614, -0.2980,  0.2887,  ...,  0.0363,  1.0205, -0.4407],
         ...,
         [-0.9115, -0.7131, -0.3658,  ...,  1.5890,  0.7217,  0.2050],
         [ 0.1728,  0.7166, -0.0298,  ..., -0.3730, -0.4352,  0.9171],
         [-2.3554, -0.7158, -1.4308,  ..., -0.3900,  0.0296, -0.2357]]],
       grad_fn=<EmbeddingBackward0>),  shape : torch.Size([1, 13, 768])
token_type_embeddings : tensor([[[-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         ...,
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.27

In [ ]:
ids = torch.tensor(tokenized_dataset[0]["input_ids"])
#print(ids.shape)
ids = ids.unsqueeze(0)
#print(ids.shape)
em = myEmbeddings(ids)

inputs_embeds : tensor([[[-0.6648,  0.4162,  0.5075,  ...,  0.0945,  1.5146,  0.3791],
         [-0.6925, -0.9747,  0.6626,  ...,  1.2095,  1.4783,  0.7562],
         [ 1.4614, -0.2980,  0.2887,  ...,  0.0363,  1.0205, -0.4407],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<EmbeddingBackward0>),  shape : torch.Size([1, 512, 768])
token_type_embeddings : tensor([[[-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         ...,
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896]]],
   

In [ ]:
em.shape

torch.Size([1, 512, 768])

In [ ]:
em[0][0].shape

torch.Size([768])

In [ ]:
em = myEmbeddings(ids)

inputs_embeds : tensor([[[-0.6648,  0.4162,  0.5075,  ...,  0.0945,  1.5146,  0.3791],
         [-0.6925, -0.9747,  0.6626,  ...,  1.2095,  1.4783,  0.7562],
         [ 1.4614, -0.2980,  0.2887,  ...,  0.0363,  1.0205, -0.4407],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<EmbeddingBackward0>),  shape : torch.Size([1, 512, 768])
token_type_embeddings : tensor([[[-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         ...,
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896],
         [-1.0806, -0.0269, -0.5304,  ...,  1.1079, -0.2740, -0.5896]]],
   

In [ ]:
class BertSelfAttention(nn.Module):
    def __init__(self, config, position_embedding_type=None):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention "
                f"heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads                                     # 12
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)           # 768 / 12 = 64
        self.all_head_size = self.num_attention_heads * self.attention_head_size                  # 12 * 64 = 768

        self.query = nn.Linear(config.hidden_size, self.all_head_size)                         # (768, 768)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)                           # (768, 768)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)                         # (768, 768)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.position_embedding_type = position_embedding_type or getattr(
            config, "position_embedding_type", "absolute"
        )
        print("position_embedding_type used : " + self.position_embedding_type)
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":                        # other type of position embedding SEE FOR LATER
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)

        self.is_decoder = config.is_decoder

    def transpose_for_scores(self, x: torch.Tensor) -> torch.Tensor:                              # To align dimensions of key query and value for the attention computation
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)       # replace last dimension with 2 dimensions
        x = x.view(new_x_shape)                                                                   # view : same data with different size
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            #print("key linear with transpose_for_scores" + str(key_layer.shape))
            #print("key linear" + str(self.key(hidden_states).shape))
            #print(" ")
            #print("key linear with transpose_for_scores" + str(value_layer.shape))
            #print("key linear" + str(self.value(hidden_states).shape))

        query_layer = self.transpose_for_scores(mixed_query_layer)
        #print("query linear with transpose_for_scores" + str(query_layer.shape))
        #print("query linear" + str(mixed_query_layer.shape))

        use_cache = past_key_value is not None
        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        #print("attention_scores : " + str(attention_scores.shape))
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":           ### FOR RELATIVE POSITION
            query_length, key_length = query_layer.shape[2], key_layer.shape[2]
            if use_cache:
                position_ids_l = torch.tensor(key_length - 1, dtype=torch.long, device=hidden_states.device).view(
                    -1, 1
                )
            else:
                position_ids_l = torch.arange(query_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(key_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r

            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key           ### FOR RELATIVE POSITION

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)
        #print("attention_probs : " + str(attention_probs.shape))
        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        #print("value linear" + str(value_layer.shape))

        context_layer = torch.matmul(attention_probs, value_layer)
        #print("attention_probs X Value : " + str(context_layer.shape))
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        #print("attention_probs X Value (PERMUTED) : " + str(context_layer.shape))
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)
        print("attention_probs X Value '(NEW): " + str(context_layer.shape))

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)
        #print("outputs : " + str(outputs.size()))

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs


In [ ]:
selfAtt = BertSelfAttention(config, position_embedding_type = None)

position_embedding_type used : absolute


In [ ]:
em

tensor([[[-1.0775,  0.0656,  0.4645,  ...,  2.2176,  1.9788,  0.1109],
         [-0.0876, -1.0523,  0.1506,  ...,  0.6166, -0.0000, -0.5556],
         [-0.5623,  0.2934,  0.0501,  ...,  0.0000,  0.6288, -0.2678],
         ...,
         [-0.4971, -0.0116, -1.5753,  ...,  1.5289, -0.8320,  0.0000],
         [ 0.0526,  1.0594, -1.5372,  ...,  0.7196, -0.3083,  0.2095],
         [ 0.6741, -0.5724, -0.3504,  ...,  0.9456, -0.7885, -0.7788]]],
       grad_fn=<MulBackward0>)

In [ ]:
selfAtt(em)

attention_probs X Value '(NEW): torch.Size([1, 512, 768])


(tensor([[[-4.1350e-02,  2.0162e-01, -6.3018e-01,  ...,  4.8795e-01,
           -7.7199e-03,  2.2495e-01],
          [-2.5686e-02,  2.1084e-01, -6.4389e-01,  ...,  4.9257e-01,
           -1.7134e-04,  2.3076e-01],
          [-2.5659e-02,  2.0909e-01, -6.6043e-01,  ...,  5.0635e-01,
            2.4485e-02,  2.2044e-01],
          ...,
          [-2.5398e-02,  2.0238e-01, -6.3443e-01,  ...,  5.0950e-01,
            8.9590e-03,  2.2946e-01],
          [-4.3894e-02,  1.8303e-01, -6.0618e-01,  ...,  4.9974e-01,
            1.1478e-02,  2.3008e-01],
          [-2.9250e-02,  1.8521e-01, -6.0847e-01,  ...,  4.6548e-01,
            2.7011e-02,  2.5434e-01]]], grad_fn=<ViewBackward0>),)

In [ ]:
class BertSelfOutput(nn.Module):                                       # add the input tensor to the context (attention output) so the model dont forget the initial input sentence
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:    # hidden_states = NEW hidden states / input_tensor = PREVIOUS hidden states
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [ ]:
out = BertSelfOutput(config)

In [ ]:
out(selfAtt(em)[0], em).shape

attention_probs X Value '(NEW): torch.Size([1, 512, 768])


torch.Size([1, 512, 768])

In [ ]:
class BertAttention(nn.Module):
    def __init__(self, config, position_embedding_type=None):
        super().__init__()
        self.self = BertSelfAttention(config, position_embedding_type=position_embedding_type)
        self.output = BertSelfOutput(config)
        self.pruned_heads = set()

    def prune_heads(self, heads):
        if len(heads) == 0:
            return
        """
        heads, index = find_pruneable_heads_and_indices(
            heads, self.self.num_attention_heads, self.self.attention_head_size, self.pruned_heads
        )

        # Prune linear layers
        self.self.query = prune_linear_layer(self.self.query, index)
        self.self.key = prune_linear_layer(self.self.key, index)
        self.self.value = prune_linear_layer(self.self.value, index)
        self.output.dense = prune_linear_layer(self.output.dense, index, dim=1)

        # Update hyper params and store pruned heads
        self.self.num_attention_heads = self.self.num_attention_heads - len(heads)
        self.self.all_head_size = self.self.attention_head_size * self.self.num_attention_heads
        self.pruned_heads = self.pruned_heads.union(heads)
        """

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        self_outputs = self.self(
            hidden_states,
            attention_mask,
            head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
        )
        attention_output = self.output(self_outputs[0], hidden_states)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs

In [ ]:
att = BertAttention(config)

position_embedding_type used : absolute


In [ ]:
att(em)

attention_probs X Value '(NEW): torch.Size([1, 512, 768])


(tensor([[[-0.9574,  0.2588,  0.5091,  ...,  2.1198,  1.8137,  0.1066],
          [-0.0571, -0.7818,  0.1959,  ...,  0.6515, -0.0057, -0.5325],
          [-0.5311,  0.4977,  0.1328,  ...,  0.0932,  0.6043, -0.2448],
          ...,
          [-0.4259,  0.2050, -1.3332,  ...,  1.4919, -0.7394, -0.0036],
          [ 0.0948,  1.1849, -1.3177,  ...,  0.7581, -0.2671,  0.2088],
          [ 0.6742, -0.3205, -0.2373,  ...,  0.9824, -0.7225, -0.7234]]],
        grad_fn=<NativeLayerNormBackward0>),)

In [ ]:
class ClassInstantier(OrderedDict):
    def __getitem__(self, key):
        content = super().__getitem__(key)
        cls, kwargs = content if isinstance(content, tuple) else (content, {})
        return cls(**kwargs)

class GELUActivation(nn.Module):
    """
    Original Implementation of the GELU activation function in Google BERT repo when initially created. For
    information: OpenAI GPT's GELU is slightly different (and gives slightly different results): 0.5 * x * (1 +
    torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3)))) This is now written in C in nn.functional
    Also see the Gaussian Error Linear Units paper: https://arxiv.org/abs/1606.08415
    """

    def __init__(self, use_gelu_python: bool = False):
        super().__init__()
        if use_gelu_python:
            self.act = self._gelu_python
        else:
            self.act = nn.functional.gelu

    def _gelu_python(self, input: Tensor) -> Tensor:
        return input * 0.5 * (1.0 + torch.erf(input / math.sqrt(2.0)))

    def forward(self, input: Tensor) -> Tensor:
        return self.act(input)

ACT2CLS = {
    "gelu": GELUActivation,
}
ACT2FN = ClassInstantier(ACT2CLS)

In [ ]:
class BertIntermediate(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = ACT2FN[config.hidden_act]
        else:
            self.intermediate_act_fn = config.hidden_act

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states

In [ ]:
Inter = BertIntermediate(config)

In [ ]:
Inter(att(em)[0])
Inter(att(em)[0]).shape

attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])


torch.Size([1, 512, 3072])

In [ ]:
class BertOutput(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states: torch.Tensor, input_tensor: torch.Tensor) -> torch.Tensor:
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [ ]:
def apply_chunking_to_forward(
    forward_fn: Callable[..., torch.Tensor], chunk_size: int, chunk_dim: int, *input_tensors
) -> torch.Tensor:
    """
    This function chunks the `input_tensors` into smaller input tensor parts of size `chunk_size` over the dimension
    `chunk_dim`. It then applies a layer `forward_fn` to each chunk independently to save memory.

    If the `forward_fn` is independent across the `chunk_dim` this function will yield the same result as directly
    applying `forward_fn` to `input_tensors`.

    Args:
        forward_fn (`Callable[..., torch.Tensor]`):
            The forward function of the model.
        chunk_size (`int`):
            The chunk size of a chunked tensor: `num_chunks = len(input_tensors[0]) / chunk_size`.
        chunk_dim (`int`):
            The dimension over which the `input_tensors` should be chunked.
        input_tensors (`Tuple[torch.Tensor]`):
            The input tensors of `forward_fn` which will be chunked

    Returns:
        `torch.Tensor`: A tensor with the same shape as the `forward_fn` would have given if applied`.


    Examples:

    ```python
    # rename the usual forward() fn to forward_chunk()
    def forward_chunk(self, hidden_states):
        hidden_states = self.decoder(hidden_states)
        return hidden_states


    # implement a chunked forward function
    def forward(self, hidden_states):
        return apply_chunking_to_forward(self.forward_chunk, self.chunk_size_lm_head, self.seq_len_dim, hidden_states)
    ```"""

    assert len(input_tensors) > 0, f"{input_tensors} has to be a tuple/list of tensors"

    # inspect.signature exist since python 3.5 and is a python method -> no problem with backward compatibility
    num_args_in_forward_chunk_fn = len(inspect.signature(forward_fn).parameters)
    if num_args_in_forward_chunk_fn != len(input_tensors):
        raise ValueError(
            f"forward_chunk_fn expects {num_args_in_forward_chunk_fn} arguments, but only {len(input_tensors)} input "
            "tensors are given"
        )

    if chunk_size > 0:
        tensor_shape = input_tensors[0].shape[chunk_dim]
        for input_tensor in input_tensors:
            if input_tensor.shape[chunk_dim] != tensor_shape:
                raise ValueError(
                    f"All input tenors have to be of the same shape: {tensor_shape}, "
                    f"found shape {input_tensor.shape[chunk_dim]}"
                )

        if input_tensors[0].shape[chunk_dim] % chunk_size != 0:
            raise ValueError(
                f"The dimension to be chunked {input_tensors[0].shape[chunk_dim]} has to be a multiple of the chunk "
                f"size {chunk_size}"
            )

        num_chunks = input_tensors[0].shape[chunk_dim] // chunk_size

        # chunk input tensor into tuples
        input_tensors_chunks = tuple(input_tensor.chunk(num_chunks, dim=chunk_dim) for input_tensor in input_tensors)
        # apply forward fn to every tuple
        output_chunks = tuple(forward_fn(*input_tensors_chunk) for input_tensors_chunk in zip(*input_tensors_chunks))
        # concatenate output at same dimension
        return torch.cat(output_chunks, dim=chunk_dim)

    return forward_fn(*input_tensors)


class BertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.chunk_size_feed_forward = config.chunk_size_feed_forward
        self.seq_len_dim = 1
        self.attention = BertAttention(config)                              # Initialize Attention
        self.is_decoder = config.is_decoder
        self.add_cross_attention = config.add_cross_attention
        if self.add_cross_attention:
            if not self.is_decoder:
                raise ValueError(f"{self} should be used as a decoder model if cross attention is added")
            self.crossattention = BertAttention(config, position_embedding_type="absolute")

        self.intermediate = BertIntermediate(config)                         # Initialize Intermediate
        self.output = BertOutput(config)                                     # Initialize Output

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask,
            head_mask,
            output_attentions=output_attentions,
            past_key_value=self_attn_past_key_value,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        if self.is_decoder:
            outputs = self_attention_outputs[1:-1]
            present_key_value = self_attention_outputs[-1]
        else:
            outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        cross_attn_present_key_value = None
        if self.is_decoder and encoder_hidden_states is not None:
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with cross-attention layers"
                    " by setting `config.add_cross_attention=True`"
                )

            # cross_attn cached key/values tuple is at positions 3,4 of past_key_value tuple
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            cross_attention_outputs = self.crossattention(
                attention_output,
                attention_mask,
                head_mask,
                encoder_hidden_states,
                encoder_attention_mask,
                cross_attn_past_key_value,
                output_attentions,
            )
            attention_output = cross_attention_outputs[0]
            outputs = outputs + cross_attention_outputs[1:-1]  # add cross attentions if we output attention weights

            # add cross-attn cache to positions 3,4 of present_key_value tuple
            cross_attn_present_key_value = cross_attention_outputs[-1]
            present_key_value = present_key_value + cross_attn_present_key_value

        layer_output = apply_chunking_to_forward(
            self.feed_forward_chunk, self.chunk_size_feed_forward, self.seq_len_dim, attention_output
        )
        outputs = (layer_output,) + outputs

        # if decoder, return the attn key/values as the last output
        if self.is_decoder:
            outputs = outputs + (present_key_value,)

        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

In [ ]:
layer = BertLayer(config)

position_embedding_type used : absolute


In [ ]:
layer(em)
layer(em)[0].shape

attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])


torch.Size([1, 512, 768])

In [ ]:
@dataclass
class BaseModelOutputWithPastAndCrossAttentions(ModelOutput):
    """
    Base class for model's outputs that may also contain a past key/values (to speed up sequential decoding).

    Args:
        last_hidden_state (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`):
            Sequence of hidden-states at the output of the last layer of the model.

            If `past_key_values` is used only the last hidden-state of the sequences of shape `(batch_size, 1,
            hidden_size)` is output.
        past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`) and optionally if
            `config.is_encoder_decoder=True` 2 additional tensors of shape `(batch_size, num_heads,
            encoder_sequence_length, embed_size_per_head)`.

            Contains pre-computed hidden-states (key and values in the self-attention blocks and optionally if
            `config.is_encoder_decoder=True` in the cross-attention blocks) that can be used (see `past_key_values`
            input) to speed up sequential decoding.
        hidden_states (`tuple(torch.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `torch.FloatTensor` (one for the output of the embeddings, if the model has an embedding layer, +
            one for the output of each layer) of shape `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the optional initial embedding outputs.
        attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
        cross_attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` and `config.add_cross_attention=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights of the decoder's cross-attention layer, after the attention softmax, used to compute the
            weighted average in the cross-attention heads.
    """

    last_hidden_state: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None
    cross_attentions: Optional[Tuple[torch.FloatTensor]] = None


In [ ]:
class BertEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.layer = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])
        self.gradient_checkpointing = False

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = False,
        output_hidden_states: Optional[bool] = False,
        return_dict: Optional[bool] = True,
    ) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPastAndCrossAttentions]:
        all_hidden_states = () if output_hidden_states else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None

        """"                                                                                      # FOR LOGGING
        if self.gradient_checkpointing and self.training:
            if use_cache:
                logger.warning_once(
                    "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                )
                use_cache = False                                                                   # FOR LOGGING
        """
        next_decoder_cache = () if use_cache else None
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            past_key_value = past_key_values[i] if past_key_values is not None else None

            if self.gradient_checkpointing and self.training:
                layer_outputs = self._gradient_checkpointing_func(
                    layer_module.__call__,
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )
            else:
                layer_outputs = layer_module(
                    hidden_states,
                    attention_mask,
                    layer_head_mask,
                    encoder_hidden_states,
                    encoder_attention_mask,
                    past_key_value,
                    output_attentions,
                )

            hidden_states = layer_outputs[0]
            if use_cache:
                next_decoder_cache += (layer_outputs[-1],)
            if output_attentions:
                all_self_attentions = all_self_attentions + (layer_outputs[1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (layer_outputs[2],)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v
                for v in [
                    hidden_states,
                    next_decoder_cache,
                    all_hidden_states,
                    all_self_attentions,
                    all_cross_attentions,
                ]
                if v is not None
            )
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=next_decoder_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )


In [ ]:
bert = BertEncoder(config)

position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute
position_embedding_type used : absolute


In [ ]:
result = bert(em)

attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])
attention_probs X Value '(NEW): torch.Size([1, 512, 768])


In [ ]:
config.num_hidden_layers

12

In [ ]:
result.shape

AttributeError: ignored

In [ ]:

class BertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output
